In [1]:
import pandas as pd

### Read In Data - Drop and Clean
___

Reading in indivudual garment datasets and combine into one dataframe.  View the data and check for nulls.

In [2]:
#Read in all of the different garment datasets - 2000 observations per csv.

hats = pd.read_csv('../data/beanie-toque_details.csv')
socks = pd.read_csv('../data/mid-calf_details.csv')
pullovers = pd.read_csv('../data/pullover_details.csv')

In [3]:
# I want to add the type of garment as a column in each dataframe.
hats['type'] = 'hat'
socks['type'] = 'socks'
pullovers['type'] = 'pullover'

In [4]:
# One observation in all of the datasets is an unusual yarn_weight.  I'll change it to 'Worsted'.
# Row is 1569
hats[hats['yarn_weight'] == 'Aran / Worsted']
hats.loc[1569, 'yarn_weight'] = 'Worsted'

In [5]:
#Combine dataframes
rav_df = pd.concat([hats, socks, pullovers], ignore_index = True)

Dropping the sizes available column.  There is no standardization of how this is populated.  It can be whatever the artist enters.  Making the assumption that most patterns are knit for most sizes or that size is specified in the name column ex. Baby Hat.  I don't believe it would help in creating a better model.

The downside to this is making an assumption that anyone using this recommender is an average size range.  Ravelry could benefit by standardizing size ranges by using either exclusively measurements or categetories.  However, this is difficult because different garments size and fit differently.

Also dropping the gauge_pattern column.  This is a value that is also set by the designer and has no standardization.  Most patterns assume gauge is measure using stockinette stitch.  After filling nulls with 'stockinette' and viewing the data, 76% of all patterns have 'stockinette' as their gauge_pattern.  I don't believe this would be helpful in pattern recommendation.

In [6]:
rav_df.drop(columns = ['sizes_available', 'gauge_pattern'], inplace = True)

In [7]:
rav_df

,id,name,author,difficulty_avg,gauge,gauge_divisor,max_yardage,notes,price,projects_count,queued_projects_count,rating_avg,yarn_weight,type
0,990044,Musselburgh,Ysolda Teague,2.46,6.0,1.0,610.0,>Our favourite swatchless hat pattern\r\n> jus...,6.00,23656,7700,4.89,Fingering,hat
1,899479,Classic Ribbed Hat,Purl Soho,1.92,32.0,4.0,305.0,MATERIALS\r\nPurl Soho’s [Cashmere Merino Bloo...,NaN,10353,5382,4.83,DK,hat
2,1353734,Alpine Bloom Hat,Caitlin Hunter,3.37,24.0,4.0,230.0,The Alpine Bloom hat is designed as a companio...,5.00,1520,2364,4.84,Sport,hat
3,528611,Classic Cuffed Hat,Purl Soho,1.87,20.0,4.0,328.0,"MATERIALS\r\n\r\n- Hat with Pom Pom: 1 (2, 2) ...",NaN,9097,5043,4.70,Worsted,hat
4,905506,February Hat,Kate Gagnon Osborn,2.62,18.0,4.0,NaN,When thinking about what I wanted to do for my...,NaN,3888,3532,4.75,Worsted,hat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,431963,Bray,Jared Flood,4.54,18.0,4.0,1595.0,> Save $1 when you purchase this pattern on [B...,13.00,345,934,4.65,Worsted,pullover
5996,507122,Barbet Turtleneck,Alexis Winslow,1.86,16.0,4.0,1736.0,THIS EASY RAGLAN PULLOVER is not only stylish ...,NaN,35,70,5.00,Worsted,pullover
5997,543012,Stonewall,Alicia Plummer,3.66,18.0,4.0,2100.0,Stonewall is an updated classic v-neck with a ...,8.00,184,907,3.93,Worsted,pullover
5998,557899,Park Pullover,Kirsten Hipsky,2.20,18.0,4.0,1288.0,UPDATE: Want more detail on how to work the ra...,5.99,39,47,4.58,Aran,pullover


### Null Cleaning
___

|Function|Argument|Function|
|---|---|---|
|**null_pipeline**|*df* - dataframe to clean|Combines all helper functions and removes all null values from dataframe|
|**gauge_calculator**|*row* -dataframe row|Inputs number of stitches per four inches depending on yarn weight|
|**yardage_calculator**|*row* - dataframe row|Inputs the average max yardage used depending on garment type|
|**stockinette_filler**|*row* - dataframe row|If there is a 'stockinette' string anywhere in the gauge_pattern column, function will replace text with only 'stockinette'.  Will also replace with 'stockinette' string if value is null|

#### Function Details
___

**null_pipeline** -- This function combines all other functions, as well as a '.fillna' line of code that removes all null values in the dataset.

**gauge_calculator** -- Gauge is typically described as number of knitting stitches per inch.  For example, a pattern with 20 in the gauge column and 4 in the gauge divisor means '20stiches per 4 inches'.  This equates to 5 stitches per inch which is indicative of a worsted weight yarn.  The gauge_calculator function is designed to fill gauge column nulls based on observation yarn weight.  Yarn weight infomation referenced from the [Craft Yarn Council](https://www.craftyarncouncil.com/standards/yarn-weight-system).

**yardage_calculator** -- The yardage and gauge functions originally were 'if' and 'elif' statements for each conditional, but they would create more nulls than fill them.  I could not figure out why and eventually copied my code into ChatGPT. It recommended doing a dictionary, which looks cleaner but wasn't in the spirit of my original code.  However, I realized that I would need different max_yardage values for each garment.  A sweater knit in worsted weight yarn takes more yarn to make than a hat.  Instead of making three different fuctions, I thought this would be a good case for a dictionary of dictionaries.  I modified ChatGPT's code into a nested dictionary so that I just require one function to fill max_values for all of my garment types.  That also meant that I could contcatenate all garment type datasets before cleaning.

**stockinette_filler** -- Gauge pattern can be anything the designer wants it to be, even a custom pattern of their own creation.  However, most of the time the gauge pattern is stockinette.  This function will look for instances of 'stockinette' in gauge pattern text and change it to only 'stockinette'.  Will also change null values in this column to stockinette.  I do not believe this would be an insightful feature in modeling, but will clean it just in case.

**.fillna values** -- The 'notes' column is optional alphanumeric information about the pattern that the designer can include with the pattern.  They also have the option of adding nothing.  Will replace nulls in notes column with 'notes not provided'.

The 'price' column nulls mean that the pattern is avaiable to download for free.  Will replace nulls in price column with 0 as to maintain float datatype. I will also round the prices to the nearest whole dollar amount.

The 'gauge_divisor column will default to 4.0 as most patterns are written as x stitches per 4 inches when referring to gauge.  The filled in gauge values also assume four inches.

In [8]:
def gauge_calculator(row):
    

    ''' 
    Meant to be applied to a garment dataframe.  Argument is 'row' and is meant to fill nulls in
    the gauge column.  Will look for any null values in the gauge columnand replace it with the
    typical 4in gauge number associated with its yarn weight.
    
    '''
    
    if pd.isnull(row['gauge']):
        yarn_weight_gauge = {
            'Worsted': 20,
            'DK': 22,
            'Bulky': 16,
            'Aran': 20,
            'Super Bulky': 12,
            'Fingering': 32,
            'Sport': 24,
            'Any gauge': 20,  # Assuming the most popular yarn type - Worsted
            'Unavailable': 20,  # Assuming worsted yarn for the reason stated above
            'Light Fingering': 32,
            'Jumbo': 5,
            'Lace': 40
        }
        
        return yarn_weight_gauge.get(row['yarn_weight'])
    else:
        return row['gauge']

In [9]:
def yardage_calculator(row):

    ''' 
    Meant to be applied to a garment dataframe.  Argument is 'row' 
    This function is meant to fill nulls the max_yardage column.  Will look for any null values in the max_yardage column
    and replace it with the average max_yardage associated with its yarn weight. Average max_yardage is based on average of all available
    max_yardage values in the dataset.
    '''
    yarn_weight_yardage = {
        'Worsted': {'hat':213.0, 'pullover':1781.0, 'socks':326.0},
        'DK': {'hat':253.0, 'pullover':1840.0, 'socks':386.0},
        'Bulky': {'hat':142.0, 'pullover':1408.0, 'socks':329.0},
        'Aran': {'hat':188.0, 'pullover':1552.0, 'socks':399.0},
        'Super Bulky': {'hat':99.0, 'pullover':1077.0, 'socks':187.0},
        'Fingering': {'hat':301.0, 'pullover':2137, 'socks':429.0},
        'Sport': {'hat':274.0, 'pullover': 2129.0, 'socks':430.0},
        'Any gauge': {'hat':259.0, 'pullover':2869.0, 'socks':450.0},  
        'Unavailable': {'hat':225.0, 'pullover':2309, 'socks':331.0}, 
        'Light Fingering': {'hat':345.0, 'pullover':2118.0, 'socks':452.0},
        'Jumbo': {'hat':124.0, 'pullover':1752.0, 'socks':264.0},
        'Lace': {'hat':250.0, 'pullover':1844.0, 'socks':610.0}
        }
    if pd.isnull(row['max_yardage']):

        return yarn_weight_yardage.get(row['yarn_weight'], {}).get(row['type'])
    else:
        return row['max_yardage']


In [10]:
# Not sure if I'll need this in the end, but will hold onto code for now
def stockinette_filler(row):
    if pd.isnull(row['gauge_pattern']):
        return 'stockinette'
    elif 'stockinette' in str(row['gauge_pattern']).lower() or 'st st' in str(row['gauge_pattern']).lower():
        return 'stockinette'
    else:
        return row['gauge_pattern']

In [11]:
def null_pipeline(df):

    # Custom functions
    df['max_yardage'] = df.apply(yardage_calculator, axis = 1)
    df['gauge'] = df.apply(gauge_calculator, axis = 1)
    
    # One-off .apply functions using values dictionary as  the .fillna reference
    # Gauge is usually described as x stitches per four inches.  This fills null gauge_divisor values
    # with 4.0.  This matches the gauge_calculator function, which fills in number of stitches per four 
    # inches depending on what garment it is.
    
    values = ({'gauge_divisor':4.0, 'notes':'no notes provided','price':0})
    df.fillna(values, inplace = True)
    
    return df

In [12]:
rav_clean_df = null_pipeline(rav_df)

In [13]:
rav_clean_df.to_csv('../data/rav_clean.csv', index = False)

In [16]:
rav_clean_df.head()

,id,name,author,difficulty_avg,gauge,gauge_divisor,max_yardage,notes,price,projects_count,queued_projects_count,rating_avg,yarn_weight,type
0,990044,Musselburgh,Ysolda Teague,2.46,6.0,1.0,610.0,>Our favourite swatchless hat pattern\r\n> jus...,6.0,23656,7700,4.89,Fingering,hat
1,899479,Classic Ribbed Hat,Purl Soho,1.92,32.0,4.0,305.0,MATERIALS\r\nPurl Soho’s [Cashmere Merino Bloo...,0.0,10353,5382,4.83,DK,hat
2,1353734,Alpine Bloom Hat,Caitlin Hunter,3.37,24.0,4.0,230.0,The Alpine Bloom hat is designed as a companio...,5.0,1520,2364,4.84,Sport,hat
3,528611,Classic Cuffed Hat,Purl Soho,1.87,20.0,4.0,328.0,"MATERIALS\r\n\r\n- Hat with Pom Pom: 1 (2, 2) ...",0.0,9097,5043,4.70,Worsted,hat
4,905506,February Hat,Kate Gagnon Osborn,2.62,18.0,4.0,213.0,When thinking about what I wanted to do for my...,0.0,3888,3532,4.75,Worsted,hat
